## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-18-02-12-05 +0000,48hills,Puff: Sampling the best of blooms at a streaml...,https://48hills.org/2025/09/puff-sampling-the-...
1,2025-09-18-02-11-00 +0000,wsj,Disney Pulls Jimmy Kimmel’s Show After Kirk Re...,https://www.wsj.com/business/media/disney-to-p...
2,2025-09-18-02-00-25 +0000,nypost,Jimmy Kimmel’s ‘unprecedented’ rejection by br...,https://nypost.com/2025/09/17/entertainment/ji...
3,2025-09-18-02-00-00 +0000,bbc,What was on the menu and who was on guest list...,https://www.bbc.com/news/articles/cn4l13gxy2lo...
4,2025-09-18-01-55-10 +0000,startribune,ABC suspends Jimmy Kimmel's late-night show in...,https://www.startribune.com/abc-suspends-jimmy...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,61
12,kirk,32
36,charlie,26
430,gaza,17
535,uk,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
346,2025-09-17-03-28-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...,161
242,2025-09-17-13-46-17 +0000,nyt,"As Trump Arrives, Europe’s Right Claims Charli...",https://www.nytimes.com/2025/09/17/world/europ...,156
289,2025-09-17-10-00-00 +0000,latimes,A Pride flag at half-staff to honor Charlie Ki...,https://www.latimes.com/california/story/2025-...,133
125,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...,126
292,2025-09-17-10-00-00 +0000,nypost,Kristi Noem reveals last text Charlie Kirk sen...,https://nypost.com/2025/09/17/us-news/kristi-n...,118


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
346,161,2025-09-17-03-28-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...
24,107,2025-09-18-01-10-17 +0000,wapo,ABC takes Jimmy Kimmel off the air over remark...,https://www.washingtonpost.com/business/2025/0...
264,79,2025-09-17-11-44-17 +0000,cbc,Israel opens new temporary route out of Gaza C...,https://www.cbc.ca/news/world/gaza-city-israel...
169,70,2025-09-17-18-21-00 +0000,wsj,Former CDC Director Susan Monarez rebutted Rob...,https://www.wsj.com/politics/policy/susan-mona...
125,65,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...
126,59,2025-09-17-20-25-00 +0000,wsj,U.S. military operations against Latin America...,https://www.wsj.com/politics/national-security...
9,59,2025-09-18-01-41-00 +0000,wsj,When the Federal Reserve cut interest rates We...,https://www.wsj.com/economy/central-banking/fe...
327,51,2025-09-17-07-09-00 +0000,nypost,Ben & Jerry’s co-founder Jerry Greenfield resi...,https://nypost.com/2025/09/17/business/ben-amp...
44,51,2025-09-18-00-00-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...
108,50,2025-09-17-21-20-00 +0000,wsj,"The Fed lowered rates by a quarter point, and ...",https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
